In [11]:
import open3d
import numpy as np
import pandas as pd
import os
import tqdm
import copy

import matplotlib.pyplot as plt
import utils.registration as registration
import utils.functions as functions
import utils.transform as transform
import utils.pointcloud as pointcloud
import utils.fread as fread
import utils.FCGF as FCGF

from utils.config import Config
from scipy.ndimage import gaussian_filter1d

In [48]:
config = Config(
    sequence_dir="data/raw_data",
    feature_dir="data/features",
    output_dir="data/trajectories/trajectory/IMU_PCD",
    experiment="exp_12",
    trial="trial_1",
    subject="subject-1",
    sequence="01",
    groundtruth_dir="data/trajectories/groundtruth",
)

config.voxel_size=0.03
config.target_fps=20
config.min_std=0.5

In [91]:
def calculate_error(config, window_size=4, duration=5):
    pose_file = os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.pose.npz")
    motion_dir = config.get_motion_dir()

    accel_df = pd.read_csv(os.path.join(motion_dir, "accel.csv"))
    gyro_df = pd.read_csv(os.path.join(motion_dir, "gyro.csv"))

    accel_df.drop_duplicates("timestamp", inplace=True)
    gyro_df.drop_duplicates("timestamp", inplace=True)
    imu_df = pd.merge(accel_df, gyro_df, on="timestamp", suffixes=("a", "g"))

    frame_rate = accel_df.shape[0] / (accel_df.timestamp.values[-1] - accel_df.timestamp.values[0]) * 1000
    win_len = int(frame_rate * window_size) # 4 seconds window

    imu_df.loc[:, "dt"] = np.concatenate([[0], (imu_df.timestamp.values[1:] - imu_df.timestamp.values[:-1]) / 1000])
    # remove first row as the dt is 0
    imu_df = imu_df.iloc[1:]
    # reset index in pandas data frame
    imu_df.reset_index(drop=True, inplace=True)

    # # gravity vector calculation    
    # gravity = imu_df.iloc[:win_len, [1, 2, 3]].mean().values
    # # removing gravity
    # imu_df[["xa", "ya", "za"]] = imu_df[["xa", "ya", "za"]] - gravity
    # Fill 0 for displacement, angles, and coordinates
    imu_df.loc[:, "x"] = np.zeros(len(imu_df))
    imu_df.loc[:, "y"] = np.zeros(len(imu_df))
    imu_df.loc[:, "z"] = np.zeros(len(imu_df))

    sigma = 2
    # apply gaussian filter to smooth acceleration and gyro data
    imu_df.loc[:, "xa"] = gaussian_filter1d(imu_df.xa.values, sigma=sigma)
    imu_df.loc[:, "ya"] = gaussian_filter1d(imu_df.ya.values, sigma=sigma)
    imu_df.loc[:, "za"] = gaussian_filter1d(imu_df.za.values, sigma=sigma)
    imu_df.loc[:, "xg"] = gaussian_filter1d(imu_df.xg.values, sigma=sigma)
    imu_df.loc[:, "yg"] = gaussian_filter1d(imu_df.yg.values, sigma=sigma)
    imu_df.loc[:, "zg"] = gaussian_filter1d(imu_df.zg.values, sigma=sigma)

    # moving average filter        
    accel_mavg = imu_df[["xa", "ya", "za"]].rolling(window=win_len).mean()
    accel_mavg.fillna(0, inplace=True)

    imu_df[["xa", "ya", "za"]] = imu_df[["xa", "ya", "za"]] - accel_mavg
    # remove the stabilization data
    imu_df = imu_df.iloc[win_len:].copy()

    # Fill 0 for displacement, angles, and coordinates
    imu_df.loc[:, "x"] = np.zeros(len(imu_df))
    imu_df.loc[:, "y"] = np.zeros(len(imu_df))
    imu_df.loc[:, "z"] = np.zeros(len(imu_df))

    # calculate displacement and rotation
    rotation_matrix = np.identity(4)

    velocity = [0, 0, 0]

    for i in range(1, len(imu_df)):
        v = imu_df.iloc[i].values
        dt = v[7]
        # current displacement and rotation
        da = np.degrees([v[j + 4] * dt for j in range(3)])
        dd = [(velocity[j] * dt) + (0.5 * v[j + 1] * dt * dt) for j in range(3)]
        
        d = np.dot(rotation_matrix, np.array([*dd, 1]))
        
        imu_df.iloc[i, 8] = imu_df.iloc[i - 1, 8] + d[0]
        imu_df.iloc[i, 9] = imu_df.iloc[i - 1, 9] + d[1]
        imu_df.iloc[i, 10] = imu_df.iloc[i - 1, 10] + d[2]
        
        rotation_matrix = transform.rotate_transformation_matrix(rotation_matrix, da[0], da[1], da[2])
        
        velocity = [velocity[j] + v[j + 1] * dt for j in range(3)]
        
    # # create trajectory as a pcd
    # xyz = imu_df.loc[:, ["x", "y", "z"]].values
    # pcd = pointcloud.make_pcd(xyz)
    # pcd.paint_uniform_color([1, 0, 0])

    # pointcloud.view(pcd)

    # load ground truth trajectory
    sequence_ts = fread.get_timstamps_from_images(config.get_sequence_dir(), ext=".depth.png")
    start_t = functions.nearest(sequence_ts, imu_df.timestamp.values[0])
    start_index = np.where(sequence_ts == start_t)[0][0]

    pose_file = os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.pose.npz")
    local_t = np.load(pose_file)["local_t"]

    trajectory_t = [np.identity(4)]

    for t in range(1, len(local_t)):
        trajectory_t.append(np.dot(trajectory_t[t - 1], local_t[t]))
        
    trajectory_t = np.array(trajectory_t)

    # pcd_gt = pointcloud.make_pcd(trajectory_t[:, :3, 3])
    # pcd_gt.paint_uniform_color([0, 1, 0])

    end_t = functions.nearest(sequence_ts, start_t + duration * 1000)
    end_index = np.where(sequence_ts == end_t)[0][0]

    gt_xyz = imu_df[imu_df.timestamp == functions.nearest(imu_df.timestamp.values, end_t)][["x", "y", "z"]].values.squeeze()
    et_xyz = trajectory_t[end_index, :3, 3]
    
    distance = np.linalg.norm(trajectory_t[start_index, :3, 3] - trajectory_t[end_index, :3, 3])

    return np.linalg.norm(gt_xyz - et_xyz), distance

In [92]:
config = Config(
    sequence_dir="data/raw_data",
    feature_dir="data/features",
    output_dir="data/trajectories/trajectory/IMU_PCD_outlier_removed_0.05",
    experiment="exp_12",
    trial="trial_1",
    subject="subject-1",
    sequence="01",
    groundtruth_dir="data/trajectories/groundtruth",
)

config.voxel_size=0.05
config.target_fps=20
config.min_std=0.5

errors = []

for trial in os.listdir(os.path.join(config.feature_dir, config.experiment)):
    config.trial = trial
    for subject in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size))):
        config.subject = subject    
        for sequence in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size), config.subject)):
            config.sequence = sequence
            print(f"Processing: {config.experiment} >> {config.trial} >> {config.subject} >> {config.sequence}")
            error, distance = calculate_error(config, window_size=4, duration=5)
            print(f"Error: {error}, Distance: {distance}")
            
            errors.append(error)

Processing: exp_12 >> trial_1 >> subject-1 >> 01
Error: 1.3128269171130882, Distance: 2.205936252691323
Processing: exp_12 >> trial_1 >> subject-1 >> 02
Error: 0.8801016141223081, Distance: 2.055345859372061
Processing: exp_12 >> trial_1 >> subject-1 >> 03
Error: 1.453528113512102, Distance: 2.082893705848942
Processing: exp_12 >> trial_1 >> subject-1 >> 04
Error: 1.0578435512201656, Distance: 1.8536494862698156
Processing: exp_12 >> trial_1 >> subject-1 >> 05
Error: 1.559756499945172, Distance: 1.7830974465647103
Processing: exp_12 >> trial_2 >> subject-1 >> 01
Error: 0.5758219004365447, Distance: 1.441923193092377
Processing: exp_12 >> trial_2 >> subject-1 >> 02
Error: 1.9788783949153614, Distance: 1.9382256289543311
Processing: exp_12 >> trial_2 >> subject-1 >> 03
Error: 2.3769754194427053, Distance: 2.0752936264266038
Processing: exp_12 >> trial_2 >> subject-1 >> 04
Error: 0.959976053895529, Distance: 1.754694449671045
Processing: exp_12 >> trial_2 >> subject-1 >> 05
Error: 0.60831

In [93]:
np.mean(errors)

1.6164014777684301

In [176]:
def calc_accuracy(config):
    estimated_pose_file = config.get_output_file(config.get_file_name() + ".npz")
    pose_file = os.path.join(config.get_groundtruth_dir(), f"{config.get_file_name()}.pose.npz")

    estimated_data = np.load(estimated_pose_file)

    feature_dir = config.get_feature_dir()
    sequence_ts = fread.get_timstamps(feature_dir, ext=".secondary.npz")
    sequence_ts = fread.sample_timestamps(sequence_ts, config.target_fps)

    target_ts = estimated_data["sequence_ts"]

    target_inds = [np.argwhere(sequence_ts == target_ts[i])[0][0] for i in range(len(target_ts))]


    groundtruth_t = np.load(pose_file)["local_t"][target_inds]
    estimated_t = estimated_data["local_t"]

    tot_correct = 0
    rot_error = []
    trans_error = []

    for i in range(len(target_ts)):
        r, t = transform.calc_error(groundtruth_t[i], estimated_t[i])
        tot_correct += (r < 2 and t < 0.1)
        rot_error.append(r)
        trans_error.append(t)
        
    accuracy = tot_correct / len(groundtruth_t)
    return accuracy, np.mean(rot_error), np.mean(trans_error)

In [178]:
config = Config(
    sequence_dir="data/raw_data",
    feature_dir="data/features",
    output_dir="data/trajectories/local/IMU_PCD_outlier_removed/",
    experiment="exp_12",
    trial="trial_1",
    subject="subject-1",
    sequence="01",
    groundtruth_dir="data/trajectories/groundtruth",
)

config.voxel_size=0.05
config.target_fps=20
config.min_std=0.5

accuracies = []
rot_error = []
trans_error = []

for trial in os.listdir(os.path.join(config.feature_dir, config.experiment)):
    config.trial = trial
    for subject in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size))):
        config.subject = subject    
        for sequence in os.listdir(os.path.join(config.feature_dir, config.experiment, config.trial, str(config.voxel_size), config.subject)):
            config.sequence = sequence
            # print(f"Processing: {config.experiment} >> {config.trial} >> {config.subject} >> {config.sequence}")
            accuracy, r, t = calc_accuracy(config)
            # print(f"Accuracy: {accuracy}, Rotation Error: {r:}, Translation Error: {t}")
            accuracies.append(accuracy)
            rot_error.append(r)
            trans_error.append(t)
            
print(np.mean(accuracies))
print(np.mean(rot_error))
print(np.mean(trans_error))

0.9851718550450925
0.42137644788716
0.04193204274222692


0.9853309640347505
0.42137644788716
0.04193204274222692